# Generating Excel Reports with Python
### (Without Ever Touching Excel)

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set default plotting parameters
plt.rcParams['figure.figsize'] = [20.0, 7.0]
plt.rcParams.update({'font.size': 22})

sns.set_palette('colorblind')
sns.set_style('white')
sns.set_context('talk')

In [3]:
# read in worker data
worker_df = pd.read_csv('data/workers.csv', parse_dates=['Hire Date'])
print(worker_df.shape)
worker_df.head()

(5000, 5)


,Worker ID,Worker Name,Hire Date,Worker Status,Team
0,1000,Paul Clark,2020-09-12,Part Time,SeaGreen
1,1001,Andrea White,2001-11-08,Full Time,Crimson
2,1002,Daniel Taylor,2002-06-10,Part Time,LightSteelBlue
3,1003,Ann Manning,2019-09-13,Per Diem,Crimson
4,1004,Brenda Gonzales,2000-12-23,Part Time,Crimson


In [4]:
# read in widget data
widget_df = pd.read_csv('data/widgets.csv')
print(widget_df.shape)
widget_df.head()

(2817555, 5)


,Item Number,Step 1,Step 2,Step 3,Worker ID
0,4411865360-1000,0.635369,4.377276,2.734590,1000
1,4411865392-1000,2.047647,3.870909,12.698644,1000
2,4411865424-1000,0.870676,6.347963,7.655127,1000
3,4411865456-1000,2.938096,3.933764,3.550750,1000
4,4411865488-1000,2.951892,3.363985,3.820368,1000


In [5]:
# group the data to find widget making statistics by worker
widgets_grouped = widget_df.groupby(['Worker ID']).agg({'Item Number':'count',
                                                        'Step 1':['mean','median','min','max'],
                                                        'Step 2':['mean','median','min','max'],
                                                        'Step 3':['mean','median','min','max']}).reset_index()
# flatten the multi-index while preserving step names
widgets_grouped.columns = [' '.join(col).strip() 
                           for col in widgets_grouped.columns.values]

# format column headers in title case
widgets_grouped.columns = widgets_grouped.columns.str.title()

# but this causes ID to be title case as well which we dont want
widgets_grouped.rename(columns={'Worker Id':'Worker ID'}, inplace=True)

widgets_grouped.head()

,Worker ID,Item Number Count,Step 1 Mean,Step 1 Median,Step 1 Min,Step 1 Max,Step 2 Mean,Step 2 Median,Step 2 Min,Step 2 Max,Step 3 Mean,Step 3 Median,Step 3 Min,Step 3 Max
0,1000,482,3.070280,2.769688,0.125459,12.444440,4.991148,4.999767,2.410102,8.145692,4.391818,3.037905,0.028713,22.908053
1,1001,767,3.055896,2.765591,0.442014,11.024066,5.027923,5.017918,2.099443,8.680220,4.042051,2.711614,0.014386,22.885953
2,1002,201,3.059709,2.741490,0.338830,12.215252,5.083601,5.081746,2.697565,7.677551,3.759197,2.601193,0.009587,16.893653
3,1003,585,3.066364,2.705782,0.175791,10.587877,5.009311,5.004230,1.498752,8.087960,3.912284,2.815251,0.003565,31.140860
4,1004,376,2.901795,2.544975,0.063468,10.953770,5.052831,5.089172,2.221984,7.636476,3.908283,2.645137,0.033438,22.389739


In [6]:
# merge dataframes together
grouped_df = pd.merge(worker_df, widgets_grouped, how='inner', on='Worker ID')
print(grouped_df.shape)
grouped_df.head()

(5000, 18)


,Worker ID,Worker Name,Hire Date,Worker Status,Team,Item Number Count,Step 1 Mean,Step 1 Median,Step 1 Min,Step 1 Max,Step 2 Mean,Step 2 Median,Step 2 Min,Step 2 Max,Step 3 Mean,Step 3 Median,Step 3 Min,Step 3 Max
0,1000,Paul Clark,2020-09-12,Part Time,SeaGreen,482,3.070280,2.769688,0.125459,12.444440,4.991148,4.999767,2.410102,8.145692,4.391818,3.037905,0.028713,22.908053
1,1001,Andrea White,2001-11-08,Full Time,Crimson,767,3.055896,2.765591,0.442014,11.024066,5.027923,5.017918,2.099443,8.680220,4.042051,2.711614,0.014386,22.885953
2,1002,Daniel Taylor,2002-06-10,Part Time,LightSteelBlue,201,3.059709,2.741490,0.338830,12.215252,5.083601,5.081746,2.697565,7.677551,3.759197,2.601193,0.009587,16.893653
3,1003,Ann Manning,2019-09-13,Per Diem,Crimson,585,3.066364,2.705782,0.175791,10.587877,5.009311,5.004230,1.498752,8.087960,3.912284,2.815251,0.003565,31.140860
4,1004,Brenda Gonzales,2000-12-23,Part Time,Crimson,376,2.901795,2.544975,0.063468,10.953770,5.052831,5.089172,2.221984,7.636476,3.908283,2.645137,0.033438,22.389739


In [7]:
# make separate dataframes based on team
# first make a list of teams
teams = list(worker_df.Team.unique())

# empty dictionary to store new dataframes in
dfs_out = {}

for team in teams:
    # filter dataframe based on team name
    tmp = grouped_df.loc[grouped_df['Team'] == team]
    print(f'{team} Team Dataframe Shape: {tmp.shape}')
    
    # add new team dataframe to dictionary
    dfs_out[team] = tmp
    

SeaGreen Team Dataframe Shape: (1257, 18)
Crimson Team Dataframe Shape: (1223, 18)
LightSteelBlue Team Dataframe Shape: (1229, 18)
MidnightBlue Team Dataframe Shape: (1291, 18)


In [8]:
# now we can access team dataframes by dictionary key
dfs_out['SeaGreen'].head()

,Worker ID,Worker Name,Hire Date,Worker Status,Team,Item Number Count,Step 1 Mean,Step 1 Median,Step 1 Min,Step 1 Max,Step 2 Mean,Step 2 Median,Step 2 Min,Step 2 Max,Step 3 Mean,Step 3 Median,Step 3 Min,Step 3 Max
0,1000,Paul Clark,2020-09-12,Part Time,SeaGreen,482,3.070280,2.769688,0.125459,12.444440,4.991148,4.999767,2.410102,8.145692,4.391818,3.037905,0.028713,22.908053
5,1005,Jenny Moore,2005-07-30,Full Time,SeaGreen,521,3.055117,2.702775,0.321208,9.910867,4.962473,4.911384,2.150133,7.984411,3.886760,2.637224,0.004128,36.422227
6,1006,Julie Moore,2008-10-28,Per Diem,SeaGreen,839,2.977169,2.607433,0.239764,9.900732,5.001293,4.976702,1.735870,8.158247,3.840518,2.842186,0.000729,24.659967
7,1007,Christina Russell,2018-03-14,Part Time,SeaGreen,424,3.023687,2.753670,0.176096,10.939197,5.000741,4.977642,2.301153,7.560065,4.012485,2.777664,0.004824,34.822143
9,1009,Austin Yates,1992-01-19,Full Time,SeaGreen,988,3.088873,2.788594,0.119788,12.442188,5.013983,5.021267,2.119497,8.144056,3.996290,2.782876,0.001342,28.684999


In [12]:
# write to excel

# variable to store filename and relative path
out_file = 'output/Widget Report by Team.xlsx'

# create pandas excel writer
# specify datetime format to ensure dates are displayed as expected in Excel
with pd.ExcelWriter(out_file,
                    datetime_format='mm/dd/yyyy') as writer:

    #loop through dictionary of dataframes and write each to it own sheet
    for sheetname, dataframe in dfs_out.items():
        # write dataframe to excel
        dataframe.to_excel(writer, sheet_name=sheetname, index=False)
        # we need the worksheet object to access XlsxWriter features
        worksheet = writer.sheets[sheetname]
        # freeze first row with our headers
        worksheet.freeze_panes(1,0)
        # add filter arrows to column headers
        # using shape to ensure we only add filters to populated columns
        worksheet.autofilter(0,0, dataframe.shape[0], dataframe.shape[1]-1)

        #determine and set column width
        for idx, col in enumerate(dataframe):
            series = dataframe[col]
            # find largest length of values and header for each column 
            max_len = max((series.astype(str).map(len).max(), #length of largest item
                          len(str(series.name)) # length of column header
                             )) + 5 # adding a little extra space
            # set column width
            worksheet.set_column(idx, idx, max_len)
